# writing data related to data ingestion

In [43]:
!wget https://github.com/entbappy/Branching-tutorial/raw/master/Chicken-fecal-images.zip

--2024-09-18 15:37:19--  https://github.com/entbappy/Branching-tutorial/raw/master/Chicken-fecal-images.zip
Resolving github.com (github.com)... 20.207.73.82
Connecting to github.com (github.com)|20.207.73.82|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/entbappy/Branching-tutorial/master/Chicken-fecal-images.zip [following]
--2024-09-18 15:37:20--  https://raw.githubusercontent.com/entbappy/Branching-tutorial/master/Chicken-fecal-images.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8001::154, 2606:50c0:8002::154, 2606:50c0:8003::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8001::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11616915 (11M) [application/zip]
Saving to: ‘Chicken-fecal-images.zip’

Chicken-fecal-image 100%[===================>]  11.08M  3.80MB/s    in 2.9s    

2024-09-18 15:37:24 (3.80 MB/s) - ‘Chi

In [1]:
import os

In [2]:
%pwd

'/home/tanay/dlops/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/tanay/dlops'

In [5]:
# creating our own datatype
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig: # this is my datatype 
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [23]:
from src.cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [32]:
class ConfigurationManager:
    def __init__(self,
     config_filepath= CONFIG_FILE_PATH, #is path pe yaml file padi h
     params_filepath= PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath) # returing a ConfigBox type
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root]) # creating directories takes list so we packed it in list
    

    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion # data ingestion part of config.yaml
        # unpacking the config
        create_directories([config.root_dir]) 
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config

# components

In [38]:
import os
import urllib.request as request
import zipfile  

from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [43]:
class DataIngestion:

    # import the configurations -> download the data -> unzip the downloaded data

    def __init__(self, config:DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL, # download the data
                filename = self.config.local_data_file # save it here
            )
            logger.info(f"{filename} downloaded with following info: {headers}")
        else:
            logger.info(f"File already exists of size {get_size(Path(self.config.local_data_file))}")

    
    def extract_zip_file(self):
        unzip_dir = self.config.unzip_dir
        os.makedirs(unzip_dir, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_dir)

In [45]:
# creating a pipeline
try:
    config= ConfigurationManager()

    data_ingestion_config = config.get_data_ingestion_config()

    data_ingestion = DataIngestion(config=data_ingestion_config)

    data_ingestion.download_file()

    data_ingestion.extract_zip_file()
    
except Exception as e:
    raise e
